##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Image Classification with Convolutional Neural Networks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/L03_image_classification_with_cnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/L03_image_classification_with_cnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this tutorial, we'll build and train a neural network to classify images of clothing, like sneakers and shirts.

## Install and import

We'll need [TensorFlow Datasets](https://www.tensorflow.org/datasets/), an API that simplifies downloading and accessing datasets, and provides several sample datasets to work with. We're also using a few helper libraries.

In [ ]:
import tensorflow as tf

In [ ]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import logging
logger = tf.get_logger()#uygulama çalışma zamanını kaydetmek ve görünütlemek için kullanıyoruz.
logger.setLevel(logging.ERROR)#logger ın hata seviyesini belirleme işlemi için kullandım.


## Import the Fashion MNIST dataset

This guide uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset, which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 $\times$ 28 pixels), as seen here:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite" width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST is intended as a drop-in replacement for the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc) in an identical format to the articles of clothing we'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

We will use 60,000 images to train and validate the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow, using the [Datasets](https://www.tensorflow.org/datasets) API:

 10 kategoriden oluşan  70.000 gri tonlamalı görüntü içeren Fashion MNIST veri kümesini kullandım.

 fashion_mnist dataseti yükleme ve bu veri kümesini eğitim doğrulama ve test setlerine bölme işlemi.

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True, split=['train[:90%]','train[90%:]', 'test'])
train_dataset, validation_dataset, test_dataset = dataset

#Veri kümesi içeriği ni öğrenmek için
print(dataset)

Notice that this time we've added a split to the training dataset, reserving 10% for use during validation.

* The model is trained using `train_dataset`.
* The model validates as it is being trained using the `validation_dataset`
* The model is tested against `test_dataset`.

The images are 28 $\times$ 28 arrays, with pixel values in the range `[0, 255]`. The *labels* are an array of integers, in the range `[0, 9]`. These correspond to the *class* of clothing the image represents:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

### Explore the data

Modeli eğitmeden önce veri kümesinin formatını inceleyelim. Aşağıda eğitim setinde 54.000 görüntü, doğrulama setinde 6000 görüntü ve test setinde 10000 görüntü olduğu gösterilmektedir:

In [ ]:
num_train_examples = len(train_dataset)
num_validation_examples = len(validation_dataset)
num_test_examples = len(test_dataset)
print("Number of training examples: {}".format(num_train_examples))
print("Number of validation examples: {}".format(num_validation_examples))
print("Number of test examples:     {}".format(num_test_examples))

## Preprocess the data

The value of each pixel in the image data is an integer in the range `[0,255]`. For the model to work properly, these values need to be normalized to the range `[0,1]`. So here we create a normalization function, and then apply it to each image in the test and train datasets.

Görüntü verilerindeki her pikselin değeri [0,255] aralığında bir tamsayıdır. Modelin düzgün çalışması için bu değerlerin [0,1] aralığına normalize edilmesi gerekir. Bu yüzden burada bir normalleştirme fonksiyonu oluşturuyoruz ve ardından bunu test ve eğitim veri kümelerindeki her bir görüntüye uyguluyoruz.

#Normalzasyon işlemi

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float64)
  images /= 255
  return images, labels


train_dataset =  train_dataset.map(normalize)
validation_dataset = validation_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)


train_dataset =  train_dataset.cache()
validation_dataset = validation_dataset.cache()
test_dataset  =  test_dataset.cache()

### Explore the processed data

Let's plot an image to see what it looks like.

In [ ]:

for image, label in test_dataset.take(10):#veri kümesinden örnek resimleri almak için.
  #break
  image = image.numpy().reshape((28,28))
  #image = image.numpy().res((30,30))


  plt.figure()
  plt.imshow(image, cmap=plt.cm.binary)
  plt.colorbar()
  plt.grid(False)
  plt.show()

Display the first 25 images from the *training set* and display the class name below each image. Verify that the data is in the correct format and we're ready to build and train the network.

Eğitim setinden ilk 50 görüntüyü görüntüleyin ve her görüntünün altında sınıf adını görüntüleyin. Verilerin doğru formatta olduğunu doğrulayın ve ağı oluşturup eğitmeye hazırız.

In [ ]:
plt.figure(figsize=(20,20)) #datasetimdeki figure size belirleme
i = 0
for (image, label) in test_dataset.take(100): #test veri setinden örnek alma işlemi.
    image = image.numpy().reshape((28,28))

    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)#grafi üzerindeki ızgarayı kapatır.
    plt.imshow(image, cmap=plt.cm.binary)
    #plt.imshow(image, cmap=plt.cm.gray)#gri tonlamalı hali

    plt.xlabel(class_names[label])
    #plt.title(class_names[label]) #yukarıdaki kodla aynı işlevi görüyor
    i += 1
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Setup the layers

The basic building block of a neural network is the *layer*. A layer extracts a representation from the data fed into it. Hopefully, a series of connected layers results in a representation that is meaningful for the problem at hand.

Much of deep learning consists of chaining together simple layers. Most layers, like `tf.keras.layers.Dense`, have internal parameters which are adjusted ("learned") during training.

For this exercise, we'll be using two new layers, the Convolution layer (`tf.keras.layers.Conv2D`) and the Max Pooling layer (`tf.keras.layers.MaxPool2D`). Refer to the slides and official documentation on how to use these layers:

* [tf.keras.layers.Conv2D reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
* [tf.keras.layers.MaxPool2D reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

**Our network layers are:**
* 2D Convolution layer - 32 filters, 3x3 kernel, ReLU activation, padding with same values
* Max pooling layer - 2x2 kernel, 2 stride
* 2D Convolution layer - 64 filters, 3x3 kernel, ReLU activation, padding with same values
* Max pooling layer - 2x2 kernel, 2 stride
* Flatten layer
* Dense layer - 128 nodes output, ReLU activation
* Dense layer - 10 nodes output, Softmax activation

#Tensorflow ile yazılmış sinir ağı modelim

In [ ]:
"""#raise Exception('This cell has a TODO task! Please complete the TODO task and remove the raise Exception statement.')

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # Giriş katmanı (28x28 piksel görüntüler için)
    tf.keras.layers.Dense(128, activation='relu'),  # Tam bağlantılı (Dense) katman,tüm nöronlar birbirlerine bağlıdır.
    tf.keras.layers.Dropout(0.2),  # Dropout katmanı,aşırı öğrenmeyi önlemek için kullanılır.
    #Rastgele seçilen nöronları devre dışı bırakarak (drop) modelin genelleştirmesini artırır.
    tf.keras.layers.Dense(10, activation='softmax')  # Çıkış katmanı
])
"""




#CNN MODEL ÖRNEĞİ

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28,1)),
    # Bu katman, 32 adet 3x3 filtre kullanarak gelen görüntü üzerinde konvolüsyon işlemi uygular ve ReLU aktivasyon fonksiyonunu kullanır.
    tf.keras.layers.MaxPooling2D((2, 2)),#Bu işlem görüntü boyutunu azaltmaya yardımcı olur.
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


#### Exercise 3.1 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/solutions/E3.1.ipynb)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation='softmax')
])

print(model)

### Exercise 3.2 Compile the model with `Model.compile`

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:

**Compile the model below with the following settings**
* *Loss function* — SparseCategoricalCrossentropy
* *Optimizer* — Adam
* *Metrics* — accuracy

Refer to the [official documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) if you've forgotten the function.

In [ ]:
#VERİ SETİ HAZIRLAMA İŞLEMİ
BATCH_SIZE = 64 #tek seferde kaçtane resmi alıp işleme soksun karar veriyorum
train_dataset = train_dataset.cache().shuffle(num_train_examples).batch(BATCH_SIZE)
validation_dataset = validation_dataset.cache().batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:

import tensorflow as tf

# Modeli tanımla
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),  # Giriş boyutunu belirt
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Modeli derle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Modeli eğitme işlemi
model.fit(train_dataset, epochs=10, validation_data=validation_dataset)


Bu çıktı, modelin eğitim sürecini gösteren eğitim geçmişi (history) bilgilerini içerir. Her bir epoch için eğitim seti ve doğrulama seti üzerindeki kayıplar (loss) ve doğruluk (accuracy) değerleri listelenmiştir. Şimdi çıktıdaki bazı önemli bilgileri açıklayalım:

Epoch Sayısı: İterasyonları temsil eder. Her epoch, modelin tüm eğitim verilerini bir kez geçmesini ifade eder.

Iterasyon Sayısı (Steps): Her bir epoch içindeki toplam iterasyon sayısını belirtir. Bu, veri setinin boyutuna ve kullanılan mini-batch boyutuna bağlıdır.

Eğitim Loss ve Accuracy: Modelin eğitim setindeki performansını gösterir. "loss" değeri, modelin eğitim setindeki kaybını, "accuracy" değeri ise doğruluğunu ifade eder.

Doğrulama (Validation) Loss ve Accuracy: Modelin doğrulama setindeki performansını gösterir. Doğrulama seti, modelin eğitim sırasında görmediği, genellikle ayrılmış bir veri setidir. "val_loss" ve "val_accuracy" değerleri, modelin doğrulama setindeki kaybını ve doğruluğunu gösterir.

Çıktıdaki örneklerde, her epoch sonunda modelin eğitim ve doğrulama setindeki performansının nasıl değiştiği görülmektedir. Özellikle eğitim setindeki kayıp ve doğruluk değerlerinin doğrulama setindeki değerlere kıyasla nasıl değiştiğine dikkat edilmelidir. Ayrıca, modelin genelleme yeteneğini değerlendirmek için doğrulama setindeki performansına odaklanmak önemlidir. Eğitim setindeki yüksek doğruluk, modelin verileri "ezberleme" eğiliminde olduğunu gösterebilir, bu nedenle doğrulama setindeki performans daha güvenilir bir gösterge olabilir.

Model eğitme işlemi

#### Exercise 3.2 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/solutions/E3.2.ipynb)

## Exercise 3.3 Train the model with `Model.fit`

First, we define the iteration behavior for the train dataset:
1. Repeat forever by specifying `dataset.repeat()` (the `epochs` parameter described below limits how long we perform training).
2. The `dataset.shuffle(dataset_size)` randomizes the order so our model cannot learn anything from the order of the examples.
3. And `dataset.batch(32)` tells `model.fit` to use batches of 32 images and labels when updating the model variables.

Training is performed by calling the `model.fit` method:
1. Feed the training data to the model using `train_dataset`.
2. The model learns to associate images and labels.
3. The `epochs=5` parameter limits training to 5 full iterations of the training dataset, so a total of 5 * 60000 = 300000 examples.


Start training the model in the code box below for **10 epochs**.

Refer to the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model) if you've forgotten the function.

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.97 (or 97%) on the training data.

### Exercise 3.3 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/solutions/E3.3.ipynb)

## Exercise 3.4 Evaluate accuracy with `Model.evaluate`

Next, compare how the model performs on the test dataset. Use all examples we have in the test dataset to assess accuracy.

Refer to the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model) on how to use the function.

In [ ]:
"""raise Exception('This cell has a TODO task! Please complete the TODO task and remove the raise Exception statement.')

# TODO - Evaluate the model"""


# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

# Print the test accuracy
print(f'Test Accuracy: {test_accuracy}')


As it turns out, the accuracy on the test dataset is smaller than the accuracy on the training dataset. This is completely normal, since the model was trained on the `train_dataset`. When the model sees images it has never seen during training, (that is, from the `test_dataset`), we can expect performance to go down.

### Exercise 3.4 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/solutions/E3.4.ipynb)

## Make predictions and explore

With the model trained, we can use it to make predictions about some images.

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape


Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [ ]:
predictions[0]

A prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different articles of clothing. We can see which label has the highest confidence value:

In [ ]:
np.argmax(predictions[0])

So the model is usually most confident that this image is a coat, or `class_names[4]`. Let's check the label:

In [ ]:
test_labels[0]

We can graph this to look at the full set of 10 class predictions

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Let's look at the 0th image, predictions, and prediction array.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

Let's plot several images with their predictions. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percent (out of 100) for the predicted label. Note that it can be wrong even when very confident.

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


Finally, use the trained model to make a prediction about a single image.

In [ ]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

`tf.keras` models are optimized to make predictions on a *batch*, or collection, of examples at once. So even though we're using a single image, we need to add it to a list:

In [ ]:
# Add the image to a batch where it's the only member.
img = np.array([img])

print(img.shape)

Now predict the image:

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` returns a list of lists, one for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [ ]:
np.argmax(predictions_single[0])

And, as before, the model predicts a label of 4 (coat).

# Exercise 3.5

Experiment with different models and see how the accuracy results differ. In particular change the following parameters:
*   Set training epochs set to 1
*   Number of neurons in the Dense layer following the Flatten one. For example, go really low (e.g. 10) in ranges up to 512 and see how accuracy changes
*   Add additional Dense layers between the Flatten and the final Dense(10), experiment with different units in these layers
*   Don't normalize the pixel values, and see the effect that has


# Exercise 3.6 - CIFAR-10 Dataset with CNNs

Let's apply what we've learned to another dataset.The [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

As our input is a colour image, we have now 3 values per pixel. When flattened, our input array is is 3072 long ($32\times32\times3$).

* What happens when you use the same network as above?
* What is the best accuracy that you can achieve?

Like in the previous lab, download, extract and load the dataset.

The extracted folder `cifar-10-batches-py` contains (in Python's pickle format):
* Training dataset: `data_batch_1 - 5`
* Test dataset: `test_batch`
* Dataset metadata: `batches.meta`

In [ ]:
import os
import glob

# Download the data
_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
zip_dir = tf.keras.utils.get_file('cifar-10-python.tar.gz', origin=_URL, extract=True)

# Get the data and meta file names
data_dir = os.path.join(os.path.dirname(zip_dir), 'cifar-10-batches-py')
train_files = glob.glob(os.path.join(data_dir,"data_batch_*"))
test_file = os.path.join(data_dir,"test_batch")
meta_file = os.path.join(data_dir,"batches.meta")

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def build_dataset(files):
    x = []
    y = []
    for file in files:
        dict = unpickle(file)
        for image in dict[b'data']:
            # Image in the dataset is stored as a 3072 length 1D array
            x.append(image)
        for label in dict[b'labels']:
            y.append(label)

    return tf.data.Dataset.from_tensor_slices((x,y))

# Build the training dataset
train_dataset  = build_dataset(train_files)

# Build the testing dataset
test_dataset = build_dataset([test_file])

# Get the metadata
meta = unpickle(meta_file)

**Now that we've got a dataset, use what you've learned in this lab to build a CNN model for classifying these images.**
* Don't forget to pre-process your data
    * You'll want change the shape of the input image from 1D to a 3D array inside your mapping function (hint: [use the reshape function](https://www.tensorflow.org/api_docs/python/tf/reshape))
    * The image is stored as `[colour channel, width, height]`, you'll need to change this odering to `[width, height, colour channel]` (hint: [use the transpose function](https://www.tensorflow.org/api_docs/python/tf/transpose))
* Remember to check our input shape as it's different from the fashion mnist dataset

In [ ]:
# Perform dataset normalisation and configure our dataset
def reshape_and_normalize(images, labels):
    # Convert from 1D array to 3D array of [3, 32, 32]
    # the image is stored as [colour channel, width, height]
    images = tf.reshape(images, (3, 32, 32))
    # Swap from [colour channel, width, height] to [width, height, colour channel]
    images = tf.transpose(images, (1, 2, 0))
    # Convert to float32
    images = tf.cast(images, tf.float32)
    # Normalize
    images /= 255
    return images, labels


train_dataset =  train_dataset.map(reshape_and_normalize)
test_dataset  =  test_dataset.map(reshape_and_normalize)

num_train_examples = 50000
BATCH_SIZE = 32
train_dataset = train_dataset.cache().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

# Create and fit our model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', 'mse'])
model.fit(train_dataset,
          epochs=2)

# Evaluate the model with the test dataset
model.evaluate(test_dataset)

### Exercise 3.6 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/python/solutions/E3.6.ipynb)

# Notice

Remember to enable GPU to make everything run faster (Runtime -> Change runtime type -> Hardware accelerator -> GPU).
Also, if you run into trouble, simply reset the entire environment and start from the beginning:
*   Edit -> Clear all outputs
*   Runtime -> Reset all runtimes